# Timeline

## 1. Überblick
Aus welchen zeitlichen Perioden liegen überhaupt Funde vor? Wie viele jeweils?

In [1]:
import utils.load_write as lw
import utils.datastructure_operations as dataops

df_horse = lw.load_csv("data/csv/abmap_horse.csv")

periods = dataops.count_amounts_in_column(df_horse, "PERIOD")
print(len(periods) , "Perioden mit der Anzahl der jeweiligen Funde: ")
dataops.pretty_print(periods)

ranges = dataops.count_amounts_in_column(df_horse, "RANGE")
print("Diese ", len(periods), " Perioden wurden weiter unterteilt in insgesamt ", len(ranges), " Zeitintervalle.")

21 Perioden mit der Anzahl der jeweiligen Funde: 
{ 'Early - Middle Iron Age': 40,
  'Early Iron Age': 26,
  'Early Medieval': 627,
  'Early Roman': 65,
  'Early Saxon': 2,
  'Iron Age': 43,
  'Late Bronze Age - Early Iron Age': 40,
  'Late Iron Age': 193,
  'Late Iron Age - Early Roman': 163,
  'Late Roman': 289,
  'Late Saxon': 41,
  'Medieval': 790,
  'Medieval - Post Medieval': 39,
  'Mid - Late Iron Age': 16,
  'Mid Roman': 10,
  'Middle Iron Age': 206,
  'Modern': 12,
  'Post Medieval': 90,
  'Roman': 383,
  'Roman - Early Medieval': 14,
  'Saxon': 10}
Diese  21  Perioden wurden weiter unterteilt in insgesamt  101  Zeitintervalle.


Bei der Betrachtung dieser Listen wird deutlich, dass einige Perioden nur sehr wenige Funde enthalten (2 für 'Early Saxon'), andere hingegen recht viele (790 für 'Medieval').
Die einzelnen Funde wurden zusätzlich zu einer Periode noch jeweils einem etwas genaueren Zeitintervall zugeordnet ('RANGE').
Insgesamt liegen 101 unterschiedliche Zeitintervalle vor, welche den insgesamt 21 Perioden zugeordnet wurden. Um einen groben zeitlichen Überblick über die Daten zu bekommen, sind dies viel zu viele Intervalle.

### Range und Periode zusammenfassen?
Im Folgenden möchte ich versuchen, die Perioden noch etwas gröber zu fassen und Attribute wie 'late' und 'early' auf die "Hauptperiode" zu reduzieren.
Dafür muss ich allerdings sicherstellen, dass sich Zeitspannen, die ich alleine vom Namen her nicht zusammenfassen würde, auch nicht überschneiden.
Außerdem wäre es hilfreich, die gesamte Spannweite der einzelnen Perioden zu kennen.

1. Liste der Perioden der Urgeschichte serialisieren, damit der lange Text nicht dauernd im Code stehen muss.

In [2]:
archaeological_periods = {'Late Bronze Age - Early Iron Age':[],
                          'Early Iron Age':[],
                          'Early - Middle Iron Age':[],
                          'Middle Iron Age':[],
                          'Iron Age':[],
                          'Mid - Late Iron Age':[],
                          'Late Iron Age':[],
                          'Late Iron Age - Early Roman':[],
                          'Early Roman':[],
                          'Mid Roman':[],
                          'Roman':[],
                          'Late Roman':[],
                          'Early Saxon':[],
                          'Saxon':[],
                          'Late Saxon':[],
                          'Roman - Early Medieval':[],
                          'Early Medieval':[],
                          'Medieval':[],
                          'Medieval - Post Medieval':[],
                          'Post Medieval':[],
                          'Modern': [],
                          }

#lw.save_json(archaeological_periods, "data/json/archaeological-periods.json")

2. Für jede der Perioden alle Range-Attribute finden. Diese bestehen aus einem Tupel $(y_s, y_e)$ mit $y_s =$ Start des Intervalls und $y_e = $ Ende des Intervalls.
Falls es mehrere Ranges für eine Periode gibt, werden diese Tupel zuerst sortiert und zwar aufsteigend nach $y_s$, um einen Überblick zu bekommen.

In [3]:
#df_horse = lw.load_csv("data/csv/abmap_horse.csv")
periods_ranges = lw.load_json("data/json/archaeological-periods.json")

"""
Get all associated ranges for each period
"""
for period, years in zip(df_horse.PERIOD, df_horse.RANGE):
    if period in periods_ranges:
        if years not in periods_ranges[period]:
            periods_ranges[period].append(years)
    else:
        periods_ranges[period] = []
        periods_ranges[period].append(years)

"""
The 'RANGE' Attribute is a string, so in order to do anything with these years we have to convert them to int.
"""
for period in periods_ranges:
    for years in periods_ranges[period]:
        periods_ranges[period][periods_ranges[period].index(years)] = dataops.range_to_tuple(years)

    periods_ranges[period].sort(key=lambda x: x[0])

#lw.save_json(periods_ranges, "data/json/archaeological-periods-w-ranges.json")

3. Anschließend wird für jede Periode aus der Liste ihrer zugehörigen range-Tupel das kleinste $y_s$ und das größe $y_e$ für die maximale range dieser Periode bestimmt, sodass $y_s = p_s =$ Start der Periode und $y_e = p_e =$ Ende der Periode.

In [4]:
import utils.load_write as lw

"""
From alle the ranges for each period find the earliest and the latest year for the min-max range
"""
period_min_max = {}
periods_ranges = lw.load_json("data/json/archaeological-periods-w-ranges.json")
for period in periods_ranges:
    period_min_max [period] = (min(periods_ranges[period],key=lambda x : x[0])[0],
                               max(periods_ranges[period],key=lambda x : x[1])[1])

#lw.save_json(period_min_max, "data/json/periods-min-max.json")

In [5]:
time_ranges_condensed = lw.load_json("data/json/time-ranges-condensed.json")